In [1]:
import duckdb

con = duckdb.connect("../data/intermediate/gdelt.db")
con.execute("SHOW TABLES").fetchdf()

,name
0,country_month_features
1,exports
2,exports_raw


In [2]:
columns = [
"GlobalEventID","Day","MonthYear","Year","FractionDate",
"Actor1Code","Actor1Name","Actor1CountryCode","Actor1KnownGroupCode",
"Actor1EthnicCode","Actor1Religion1Code","Actor1Religion2Code",
"Actor1Type1Code","Actor1Type2Code","Actor1Type3Code",
"Actor2Code","Actor2Name","Actor2CountryCode","Actor2KnownGroupCode",
"Actor2EthnicCode","Actor2Religion1Code","Actor2Religion2Code",
"Actor2Type1Code","Actor2Type2Code","Actor2Type3Code",
"IsRootEvent","EventCode","EventBaseCode","EventRootCode",
"QuadClass","GoldsteinScale","NumMentions","NumSources",
"NumArticles","AvgTone",
"Actor1Geo_Type","Actor1Geo_FullName","Actor1Geo_CountryCode",
"Actor1Geo_ADM1Code","Actor1Geo_ADM2Code","Actor1Geo_Lat",
"Actor1Geo_Long","Actor1Geo_FeatureID",
"Actor2Geo_Type","Actor2Geo_FullName","Actor2Geo_CountryCode",
"Actor2Geo_ADM1Code","Actor2Geo_ADM2Code","Actor2Geo_Lat",
"Actor2Geo_Long","Actor2Geo_FeatureID",
"ActionGeo_Type","ActionGeo_FullName","ActionGeo_CountryCode",
"ActionGeo_ADM1Code","ActionGeo_ADM2Code","ActionGeo_Lat",
"ActionGeo_Long","ActionGeo_FeatureID",
"DATEADDED","SOURCEURL"
]

column_string = ", ".join(columns)

con.execute(f"""
CREATE OR REPLACE TABLE exports AS
SELECT *
FROM read_csv_auto(
    '../data/raw/raw_exports/*.CSV',
    delim='\t',
    header=False
) AS t({column_string})
""")


In [5]:
con.execute("DESCRIBE exports").fetchdf()

,column_name,column_type,null,key,default,extra
0,GlobalEventID,BIGINT,YES,None,None,None
1,Day,BIGINT,YES,None,None,None
2,MonthYear,BIGINT,YES,None,None,None
3,Year,BIGINT,YES,None,None,None
4,FractionDate,DOUBLE,YES,None,None,None
...,...,...,...,...,...,...
56,ActionGeo_Lat,DOUBLE,YES,None,None,None
57,ActionGeo_Long,DOUBLE,YES,None,None,None
58,ActionGeo_FeatureID,VARCHAR,YES,None,None,None
59,DATEADDED,BIGINT,YES,None,None,None


In [7]:
con.execute("""
SELECT 
    COUNT(*) as total_rows,
    MIN(Year) as min_year,
    MAX(Year) as max_year
FROM exports
""").fetchdf()

,total_rows,min_year,max_year
0,7794077,2015,2026


In [8]:
con.execute("""
SELECT
    COUNT(*) as total,
    COUNT(Actor1CountryCode) as actor1_not_null,
    COUNT(Actor2CountryCode) as actor2_not_null,
    COUNT(ActionGeo_CountryCode) as actiongeo_not_null
FROM exports
""").fetchdf()

,total,actor1_not_null,actor2_not_null,actiongeo_not_null
0,7794077,4438298,3500521,7576035
